In [1]:
from pyspark.sql import SparkSession

In [5]:
spark = ( 
 SparkSession
 .builder
    .master("local[*]")
 .appName('spark_sql')
 .getOrCreate()
)


In [6]:
df = (
    spark
    .read
    .option('delimiter', ';')
    .option('header', 'true')
    .option('inferSchema', 'true')
    .option('enconding', 'ISO-8859-1')
    .csv('./dados/precos-gasolina-etanol-10.csv')
)


In [17]:
df.printSchema()


root
 |-- Regiao - Sigla: string (nullable = true)
 |-- Estado - Sigla: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Revenda: string (nullable = true)
 |-- CNPJ da Revenda: string (nullable = true)
 |-- Nome da Rua: string (nullable = true)
 |-- Numero Rua: string (nullable = true)
 |-- Complemento: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Data da Coleta: string (nullable = true)
 |-- Valor de Venda: string (nullable = true)
 |-- Valor de Compra: string (nullable = true)
 |-- Unidade de Medida: string (nullable = true)
 |-- Bandeira: string (nullable = true)



In [19]:
df.createOrReplaceTempView('combustiveis')

In [28]:
spark.sql("""
    select `Estado - Sigla`, `Produto`,`Valor de Venda`, `Unidade de Medida`from combustiveis 
""").show(5)


+--------------+------------------+--------------+-----------------+
|Estado - Sigla|           Produto|Valor de Venda|Unidade de Medida|
+--------------+------------------+--------------+-----------------+
|            AL|          GASOLINA|          6,08|       R$ / litro|
|            AL|GASOLINA ADITIVADA|          6,08|       R$ / litro|
|            AL|            ETANOL|          4,78|       R$ / litro|
|            AL|          GASOLINA|          5,79|       R$ / litro|
|            AL|            ETANOL|          4,29|       R$ / litro|
+--------------+------------------+--------------+-----------------+
only showing top 5 rows



In [29]:
spark.sql("""
    select * from combustiveis where `Valor de Compra` IS NOT NULL
""").show()


+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+
|Regiao - Sigla|Estado - Sigla|Municipio|Revenda|CNPJ da Revenda|Nome da Rua|Numero Rua|Complemento|Bairro|Cep|Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|Bandeira|
+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+
+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+



In [30]:
view_precos = spark.sql("""
    select 
        `Estado - Sigla`, 
        `Produto`, 
        regexp_replace(`Valor de Venda`, ",", ".") as `Valor de Venda`,
        `Unidade de Medida`
    from combustiveis
""")

In [33]:
view_precos.show(5)

+--------------+------------------+--------------+-----------------+
|Estado - Sigla|           Produto|Valor de Venda|Unidade de Medida|
+--------------+------------------+--------------+-----------------+
|            AL|          GASOLINA|          6.08|       R$ / litro|
|            AL|GASOLINA ADITIVADA|          6.08|       R$ / litro|
|            AL|            ETANOL|          4.78|       R$ / litro|
|            AL|          GASOLINA|          5.79|       R$ / litro|
|            AL|            ETANOL|          4.29|       R$ / litro|
+--------------+------------------+--------------+-----------------+
only showing top 5 rows



In [36]:
df_diferenca_precos = spark.sql("""
    select 
        `Estado - Sigla`,
        `Produto`,
        `Unidade de Medida`,
        MIN(`Valor de Venda`) as menor_valor,
        MAX(`Valor de Venda`) as maior_valor,
        MAX(`Valor de Venda`) - MIN(`Valor de Venda`) as diferenca
   from view_precos
   group by all
   order by diferenca DESC
""")
df_diferenca_precos.show(5)

+--------------+------------------+-----------------+-----------+-----------+------------------+
|Estado - Sigla|           Produto|Unidade de Medida|menor_valor|maior_valor|         diferenca|
+--------------+------------------+-----------------+-----------+-----------+------------------+
|            SP|GASOLINA ADITIVADA|       R$ / litro|       4.79|       8.69|3.8999999999999995|
|            SP|            ETANOL|       R$ / litro|       2.78|       6.19|3.4100000000000006|
|            SP|          GASOLINA|       R$ / litro|       4.59|       7.59|               3.0|
|            PA|            ETANOL|       R$ / litro|       3.85|        6.6|2.7499999999999996|
|            RS|            ETANOL|       R$ / litro|       3.88|       6.29|              2.41|
+--------------+------------------+-----------------+-----------+-----------+------------------+
only showing top 5 rows

